1\. rozglądam sie po systemie, sprawdzam użytkowników, bazy

In [1]:
use master;
select [name] from master.sys.databases;
-- select [name] from master.sys.sql_logins;

(4 rows affected)

Total execution time: 00:00:00.177

name
master
tempdb
model
msdb


2\. zakładam bazę

In [3]:
IF EXISTS (SELECT [name] FROM master.sys.sysdatabases WHERE name = 'DB_OKNO')
    BEGIN
    PRINT 'baza DB_OKNO istnieje';    
    DROP DATABASE DB_OKNO;
    PRINT 'baza usunieta';
    END

CREATE DATABASE DB_OKNO 
    ON PRIMARY (
        name=okno,
        filename='/var/opt/mssql/data/okno.mdf',
        size=20MB,
        maxsize=50MB,
        filegrowth=10MB
    )
    LOG ON (
        name=okno_log,
        filename='/var/opt/mssql/data/okno.ldf',
        size=20MB,
        maxsize=50MB,
        filegrowth=10MB
    )
    COLLATE POLISH_100_CI_AS_SC_UTF8
    GO
    

    

baza DB_OKNO istnieje

baza usunieta

Total execution time: 00:00:03.541

dodaję tabele

In [58]:
use DB_OKNO
GO

/* *** slowniki *** */
DROP TABLE IF EXISTS lang;
DROP TABLE IF EXISTS issue;
DROP TABLE IF EXISTS code;
DROP TABLE IF EXISTS category;

go
CREATE TABLE category (
    ID_category DECIMAL(10) NOT NULL ,
    category_name varchar(32)

    PRIMARY KEY ( ID_category )

);
go
/*
INSERT INTO category VALUES( 10000 , 'Kosiarki'); 
INSERT INTO category VALUES( 10001 , '- Kosiarki spalinowe'); 
INSERT INTO category VALUES( 10002 , '- Kosiarki elektryczne'); 
*/



CREATE TABLE code (
    ID_code BIGINT NOT NULL IDENTITY ,
    code_string varchar(32),
    EAN CHAR(13), -- po EAN (w tym rozwiazaniu) nie wyszukujemy, są EAN zaczynające się od 0 dlatego CHAR a nie BIGINT/DECIMAL(13)
    active BIT NOT NULL DEFAULT 1,
    category DECIMAL(10),

    PRIMARY KEY ( ID_code ),
    FOREIGN KEY ( category ) REFERENCES category( ID_category )
    ON DELETE CASCADE
    ON UPDATE CASCADE
);
go

/*
INSERT INTO code ( code_string ) VALUES ( '80C051' );  
INSERT INTO code ( code_string ) VALUES ( '80C052' );

INSERT INTO code ( code_string ) VALUES ( '0602'); 
INSERT INTO code ( code_string ) VALUES ( '0603');  

INSERT INTO code ( code_string ) VALUES ( 'DED780911');  
INSERT INTO code ( code_string ) VALUES ( 'DED780912');  
INSERT INTO code ( code_string ) VALUES ( 'DED780920');
*/



CREATE TABLE issue (
    ID_issue BIGINT NOT NULL IDENTITY ,
    issue_name varchar(255) NOT NULL 

    PRIMARY KEY ( ID_issue )
);
go

CREATE TABLE lang (
    ID_lang BIGINT NOT NULL,
    lang_code char(2) NOT NULL,
    currency  nchar(2) NOT NULL,
    zeros   int NOT NULL,
    table_header nvarchar(255),

    PRIMARY KEY ( ID_lang )
);
go

    INSERT INTO lang VALUES (1,'PL','zł',2,'nr. kat.|%|cena|promocja');
    INSERT INTO lang VALUES (2,'CZ','Kč',1,'kat. č.|popis|cena|akční cena');
    INSERT INTO lang VALUES (3,'EN','€',2, 'cat. no.|description|price|promo price');
    INSERT INTO lang VALUES (4,'HU','Ft',0,'cikkszám|leírás|ár|akciós ár');
    -- ****
    INSERT INTO lang VALUES (15,'UA','€',2,'Код|опис|ціна|акційна ціна');


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.083